In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 30 kB/s 
     |████████████████████████████████| 198 kB 52.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=25d547f7c908f186fd1c1636efa6b727a11257a48c0c408e85deb307f550afde
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
!nohup ssh -o StrictHostKeyChecking=no -R mars.ru77.ru:40014:*:4040 aig@mars.ru77.ru -p 2222 &

nohup: appending output to 'nohup.out'


In [ ]:
import pyspark

sc = pyspark.SparkContext()

In [ ]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
! wget "http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz" -O "./kddcup.data_10_percent.gz"

--2022-05-24 15:59:37--  http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2144903 (2.0M) [application/x-gzip]
Saving to: ‘./kddcup.data_10_percent.gz’

./kddcup.data_10_pe 100%[===================>]   2.04M  9.22MB/s    in 0.2s    

2022-05-24 15:59:38 (9.22 MB/s) - ‘./kddcup.data_10_percent.gz’ saved [2144903/2144903]



In [ ]:
import os
data_path = 'kddcup.data_10_percent.gz'
raw_data = sc.textFile('file://' + os.path.abspath(os.curdir) + '/' + data_path)

In [ ]:
type(raw_data)

pyspark.rdd.RDD

In [ ]:
raw_data.count()

494021

In [ ]:
raw_data.take(5)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,29,29,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,39,39,1.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.00,0.00,0.00,0.00,1.00,0.00,0.00,49,49,1.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,normal.']

In [ ]:
memrdd = sc.parallelize([1,2,3,4])
print(memrdd.count())
print(memrdd.take(3))

4
[1, 2, 3]


In [ ]:
filtered = raw_data.filter(lambda x: "normal." in x)
filtered.count()

97278

In [ ]:
csv_data = raw_data.map(lambda x: x.split(","))

In [ ]:
csv_data.first()

In [ ]:
def parse(x):
    value = x.split(",")
    key = value[-1]
    return key, value

with_key = raw_data.map(parse)

In [ ]:
with_key.first()

In [ ]:
def extract_key(x):
    value = x.split(",")
    return f"{value[1]}_{value[2]}", value

key_csv = raw_data.map(extract_key)

In [ ]:
key_csv.countByKey()

In [ ]:
texts = ['Apache Spark has as its architectural foundation the resilient distributed dataset RDD a read only multiset of data items distributed over a cluster of machines that is maintained in a fault tolerant way 2 The Dataframe API was released as an abstraction on top of the RDD followed by the Dataset API In Spark 1 x the RDD was the primary application programming interface API but as of Spark 2 x use of the Dataset API is encouraged 3 even though the RDD API is not deprecated 4 5 The RDD technology still underlies the Dataset API 6 7 ',
 'Spark and its RDDs were developed in 2012 in response to limitations in the MapReduce cluster computing paradigm which forces a particular linear dataflow structure on distributed programs MapReduce programs read input data from disk map a function across the data reduce the results of the map and store reduction results on disk Spark s RDDs function as a working set for distributed programs that offers a deliberately restricted form of distributed shared memory 8 ',
 'Spark facilitates the implementation of both iterative algorithms which visit their data set multiple times in a loop and interactive exploratory data analysis i e the repeated database style querying of data The latency of such applications may be reduced by several orders of magnitude compared to Apache Hadoop MapReduce implementation 2 9 Among the class of iterative algorithms are the training algorithms for machine learning systems which formed the initial impetus for developing Apache Spark 10 ',
 'Apache Spark requires a cluster manager and a distributed storage system For cluster management Spark supports standalone native Spark cluster Hadoop YARN or Apache Mesos 11 For distributed storage Spark can interface with a wide variety including Alluxio Hadoop Distributed File System HDFS 12 MapR File System MapR FS 13 Cassandra 14 OpenStack Swift Amazon S3 Kudu or a custom solution can be implemented Spark also supports a pseudo distributed local mode usually used only for development or testing purposes where distributed storage is not required and the local file system can be used instead in such a scenario Spark is run on a single machine with one executor per CPU core ',
 'Spark Core',
 'Spark Core is the foundation of the overall project It provides distributed task dispatching scheduling and basic I O functionalities exposed through an application programming interface for Java Python Scala and R centered on the RDD abstraction the Java API is available for other JVM languages but is also usable for some other non JVM languages such as Julia 15 that can connect to the JVM This interface mirrors a functional higher order model of programming a driver program invokes parallel operations such as map filter or reduce on an RDD by passing a function to Spark which then schedules the function s execution in parallel on the cluster 2 These operations and additional ones such as joins take RDDs as input and produce new RDDs RDDs are immutable and their operations are lazy fault tolerance is achieved by keeping track of the lineage of each RDD the sequence of operations that produced it so that it can be reconstructed in the case of data loss RDDs can contain any type of Python Java or Scala objects ',
 'Besides the RDD oriented functional style of programming Spark provides two restricted forms of shared variables broadcast variables reference read only data that needs to be available on all nodes while accumulators can be used to program reductions in an imperative style 2 ',
 'A typical example of RDD centric functional programming is the following Scala program that computes the frequencies of all words occurring in a set of text files and prints the most common ones Each map flatMap a variant of map and reduceByKey takes an anonymous function that performs a simple operation on a single data item or a pair of items and applies its argument to transform an RDD into a new RDD '
]

text_rdd = sc.parallelize(texts)

In [ ]:
text_rdd.first()

'Apache Spark has as its architectural foundation the resilient distributed dataset RDD a read only multiset of data items distributed over a cluster of machines that is maintained in a fault tolerant way 2 The Dataframe API was released as an abstraction on top of the RDD followed by the Dataset API In Spark 1 x the RDD was the primary application programming interface API but as of Spark 2 x use of the Dataset API is encouraged 3 even though the RDD API is not deprecated 4 5 The RDD technology still underlies the Dataset API 6 7 '

In [ ]:
word_count = text_rdd.flatMap(lambda x: x.lower().split()).map(lambda x: (x, 1)).reduceByKey(lambda a, b: a+b)

In [ ]:
assert dict(word_count.collect())['spark'] == 17